In [7]:
import bqplot.pyplot as pl
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Accordion, IntSlider, FloatSlider, HTMLMath, Dropdown, Box, HBox, VBox, Layout, Play, jslink
from IPython.display import display, HTML, Video, Audio
import time
import playsound




import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
import math

In [9]:
# Audio("cash_register.mp3",autoplay=True)
playsound.playsound("cash_register.mp3", block=True)

ModuleNotFoundError: No module named 'gi'

In [2]:
plot_windmill = pl.figure()

In [3]:
pl.clear()
# Set background color
#plt.style.use('dark_background') # Global background color of the figure

# Define some custom colors
nice_blue0 = [ 0.74800463,  0.82674357,  0.9015917,   1.  ]
nice_blue = [ 0.54901963,  0.58559018,  0.77517879,  1. ]
nice_blue1 = np.array([ 135,  170,  222,  255. ])
nice_blue1/=255
gold = "#ffff33"
gold1 = "#ffdd55ff"
my_color = gold1
textcolor=nice_blue
facecolor="black"

number_of_points = 10
# Create random coordinates
pts = np.random.uniform(-8, 8, size=(number_of_points,2))
# pts = np.array([[0,0],[6,6],[6,-1]])
# Sort list by increasing y-axis number: https://stackoverflow.com/questions/2828059/sorting-arrays-in-numpy-by-column
#pts = pts[pts[:,1].argsort()]

# Define initial pivot point as first point of the list
pivot = pts[0]
# Define a "fake" previous pivot, located at the right of the current pivot
previous_pivot = np.array([pivot[0]+1,pivot[1]])

# Calculates the angle between a horizontal line and PM: OPM
#    M
#   /
#  / )
# P-----O
def calc_angle(P,M):
    hori = np.array([1,0])
    #vector1 = P - previous_P
    vector1 = hori
    vector2 = M - P
    #print(M,P,vector1,vector2)
    v1v2 = np.dot(vector1,vector2)
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    angle = np.arccos(v1v2/(norm1*norm2))
    # We want our angle to be always in [0,2pi]
    if(vector1[0]*vector2[1] - vector1[1]*vector2[0] < 0):
        angle = 2*np.pi - angle

    return angle

# Find next pivot based on previous pivot
def next_pivot(previous_P,P,pts,counterclockwise):
    index = 0
    min_angle = 1e6
    angle = 1e6
    piv_next = P
    # Calculates angle between current and former pivot
    angle1 = calc_angle(P,previous_P)

    # Determine index of current pivot (in the list of points)
    for i_,pt in enumerate(pts):
        if ( (P==pt).all() ):
            index = i_

    # Loop through all points and find which point gives the smallest angle difference between P--previous_P and P--pt.
    # The one yielding smallest difference will be the next pivot
    for i_,pt in enumerate(pts):
        if ( (P!=pt).any() ): # Exclude current pivot
            # Calculate angle between current pivot and another point
            angle2 = calc_angle(P,pt)
            # Make a list of all the possible angle differences that exist between the 2 lines (there are 4), modulo 2pi
            # This is because since we have an infinite line, the next pivot can be touched from the "front" or the "rear" of the line,
            # hence it's like each point has a virtual counterpart with an angle incremented by +pi.
            lst =np.array([angle2-angle1,angle2-(math.fmod(angle1+np.pi,2*np.pi)), math.fmod(angle2+np.pi,2*np.pi) -angle1 ,(math.fmod(angle2+np.pi,2*np.pi))-math.fmod(angle1+np.pi,2*np.pi)])
            if (counterclockwise):
            # Take the minimal positive difference (in our case the windmill is rotating counterclockwise, so the diff has to be positive)
                angle = min([n for n in lst  if n>0])
                # Assign current point to next pivot if conditions are met
                if (angle < min_angle and angle > 0):
                    min_angle = angle
                    piv_next = pt
            else:
                # Take the max negative difference (in this case the windmill is rotating clockwise, so the diff has to be negative)
                angle = max([n for n in lst  if n<0])
                # Assign current point to next pivot if conditions are met
                if (abs(angle) < abs(min_angle) and angle < 0):
                    min_angle = angle
                    piv_next = pt

    return piv_next,min_angle,index


# Define the rotation the line has to make to reach next pivot
current_angle = 0
old_rot_deg = 0
counterclockwise = True
old_counterclockwise = True

# Set up plot
axes_options = {'x': {'visible': False}, 'y': {'visible': False}}
## adding grid lines and changing the side of the axis in the figure above

nice_blue = [ 0.54901963,  0.58559018,  0.77517879,  1. ]

# The line is infinite, so it needs to have its limits always beyond the window dimensions
line = pl.plot([pivot[0]-30,pivot[0]+30], [pivot[1],pivot[1]], lw=1.5, colors = ["#6699cc"],axes_options=axes_options)

#scatt = plt.scatter(pts[:,0],pts[:,1],s=60,edgecolor=nice_blue1,color=nice_blue1,facecolor='none')
#scatt = plt.scatter(pts[:,0],pts[:,1],s=60,edgecolor=nice_blue1,color=nice_blue1)
scatt = pl.scatter(pts[:,0],pts[:,1], default_size=150,colors=["white"],stroke='black',names=[0 for j,_ in enumerate(pts)],names_unique=False)
#scatt = pl.scatter([1,2,3,4],[5,6,7,8], default_size=150,colors=["white"],stroke='black',names=[0,0,0,0],names_unique=False)
#scatt = plt.plot(pts[:,0],pts[:,1],'o',color=nice_blue1)
# Define a scatter plot that will only keep track of the current pivot
scatt_piv = pl.scatter([],[],default_size=150,colors=["orange"],stroke='black')

pl.xlim(-15,15)
pl.ylim(-15,15)
pl.axes(options={'x': {'grid_lines': 'none'}, 'y': {'grid_lines': 'none'}})

# pl.show()

# Create annotations for each point that will indicate the number of times a pivot has been touched. Initialized to 0.
#annotations = []
#for i_ in range(number_of_points):
#    annotations.append(plt.annotate("0",(pts[i_,0]-0.3,pts[i_,1]+0.6),color=nice_blue0))

#print("annot",annotations,annotations[0].get_text())
# Find possible options for updating "annotations"
#test = [x for x in dir(scatt)]
#print(test)
#print(scatt.names)
#scatt.names[0]+=1
#print(scatt.names)

#txt = plt.text(12, 14, "", fontsize=9)



{'x': Axis(grid_lines='none', scale=LinearScale(max=15.0, min=-15.0), visible=False),
 'y': Axis(grid_lines='none', orientation='vertical', scale=LinearScale(max=15.0, min=-15.0), visible=False)}

In [4]:
# Function to define when blit=True
# Create background of the animation that will be present on each image
def init():
    line.set_data([],[])
    return line,

# Define animation function that updates the plot
def animate_curve(rot_deg): 
    
    global pts
    global previous_pivot
    global pivot
    global min_angle
    global current_angle
    global old_rot_deg
    global old_counterclockwise
    #global txt
   
    # Determines sense of rotation
    counterclockwise = (rot_deg - old_rot_deg >= 0)
    # Calculates the coordinates of a point along a circle centered around the current pivot point
    # We use polar coordinates
    #angle_widget.value = int(true_angle_widget.value)
    index = 0
    radius = 30 # Needs to be larger than the dimensions of the plot window
    dangle = 0.1 # Incremental angle, in degrees
#     angle_widget.value=rot_deg
    theta = math.radians(rot_deg) # Converts to radians
    # Define coordinates. Note that the center is not (0,0), but instead the current pivot, so we add its coordinates
    x = radius*math.cos(theta) + previous_pivot[0] 
    y = radius*math.sin(theta) + previous_pivot[1]
    # Create symmetric point with respect to pivot, just to draw a line that extends on both sides
    x2 = -radius*math.cos(theta) + previous_pivot[0] 
    y2 = -radius*math.sin(theta) + previous_pivot[1]
    # Update line coordinates 
    line.x = [x2,x]
    line.y = [y2,y]
    #txt.set_text("Rotation: "+str(round(math.degrees(math.fmod(theta,2*np.pi)),1))+"°")
    
    if (counterclockwise != old_counterclockwise):
#         print("CHANGE")
        new_pivot, min_angle, index = next_pivot(pivot,previous_pivot,pts,counterclockwise)
#         previous_pivot = pivot
        pivot = new_pivot
        current_angle = current_angle + min_angle
#         print("CHANGE_END", previous_pivot,pivot,min_angle)
    
    # When the current moving line reaches the limit angle, determines next pivot
    # Case when the line is moving counterclockwise
    if (counterclockwise and (rot_deg + dangle >= math.degrees(current_angle)) ):
        # Highlight newly touched pivot
        scatt_piv.x = [pivot[0]]
        scatt_piv.y = [pivot[1]]
        # Calculates next pivot
        new_pivot, min_angle, index = next_pivot(previous_pivot,pivot,pts,counterclockwise)
        previous_pivot = pivot
        pivot = new_pivot
        # Determines which angle (radians) the line has to reach until next pivot
        current_angle = current_angle + min_angle # Here the errors stack up...
        # Get how many times the pivot has been visited and increment it by 1.
        lst = scatt.names.copy()
        lst[index]+=1
        scatt.names=lst
#         print("CURRENT_COUNTERCLOCK",current_angle)
    # Case when the line is rotating clockwise
    elif ( (not counterclockwise) and (rot_deg - dangle <= math.degrees(current_angle)) ):
#         print("hmm",previous_pivot,pivot)
        # Highlight newly touched pivot
        scatt_piv.x = [pivot[0]]
        scatt_piv.y = [pivot[1]]
        # Calculates next pivot
        new_pivot, min_angle, index = next_pivot(previous_pivot,pivot,pts,counterclockwise)
        previous_pivot = pivot
        pivot = new_pivot
        # Determines which angle (radians) the line has to reach until next pivot
        current_angle = current_angle + min_angle # Here the errors stack up...
        # Get how many times the pivot has been visited and increment it by 1.
        lst = scatt.names.copy()
        lst[index]+=1
        scatt.names=lst
#         print("CURRENT_CLOCK",current_angle)
        
    old_rot_deg = rot_deg
    old_counterclockwise = counterclockwise

    return line,scatt_piv,#annotations[index],


#ani_curve = animation.FuncAnimation(fig, animate_curve, init_func=init, frames=1000, blit=True, interval=10, repeat=False)
#ani_curve = animation.FuncAnimation(fig, animate_curve, init_func=init, frames=1000, blit=False, interval=10, repeat=False)

# Show animation
#plt.show()

# Define movie settings
#FFMpegWriter = animation.writers['ffmpeg']
#metadata = dict(title='windmill_problem', artist='leinahtan',
#                comment='Replicate 3Blue1Brown')

# Change the video bitrate as you like and add some metadata.
#mywriter = FFMpegWriter(fps=50, bitrate=500, metadata=metadata)

#Save animation
#my_dpi=300
#ani_curve.save("mathematical_windmill.mp4",writer=mywriter,dpi=my_dpi)



In [5]:
#HTML(ani_curve.to_html5_video()) #First produces video and then plays it
#Video("mathematical_windmill.mp4")
# HTML("""
#     <video width="430" height="360" alt="test" controls>
#         <source src="mathematical_windmill.mp4" type="video/mp4">
#     </video>
# """)

In [6]:
from ipywidgets import Accordion, IntSlider, FloatSlider, HTMLMath, Dropdown, Box, HBox, VBox, Layout
from IPython.display import display
angle_widget = widgets.IntSlider(min=0, max=7200, step=10, value=0, description = "$\\theta$", continuous_update=True)
true_angle_widget = widgets.FloatSlider(min=0., max=720., step=0.1, value=0., description = "$\\theta$°", continuous_update=True)
type_widget = Dropdown(options=(
        ("Random","random"),
        ("Consecutive","consecutive"),
        ("Manual","manual"),    
    ), 
    description = "Selection type", continuous_update=False, layout=Layout(width='250px'))
button_clear = widgets.Button(description="Clear selection!")
button_run = widgets.Button(description="Run")
output = widgets.Output()

def on_button_clear_clicked(b):
    global current_angle
    global previous_pivot
    global pivot
    global old_counterclockwise
    global counterclockwise
    
    with output:
        #print("Button clicked.")
        counterclockwise = True
        old_counterclockwise = True
        # Define initial pivot point as first point of the list
        pivot = pts[0]
        # Define a "fake" previous pivot, located horizontally at the right side of the current pivot
        previous_pivot = np.array([pivot[0]+1,pivot[1]])
        scatt.names = [0 for j,_ in enumerate(pts)]
        line.x = [pivot[0]-30,pivot[0]+30]
        line.y = [pivot[1],pivot[1]]
        current_angle = 0
        scatt_piv.x = [pivot[0]]
        scatt_piv.y = [pivot[1]]
#         true_angle_widget.unobserve(on_change0, names='value', type='change')
        true_angle_widget.value = 0
#         true_angle_widget.observe(on_change0, names='value', type='change')
        angle_widget.unobserve(on_change1, names='value', type='change')
        angle_widget.value = 0
        angle_widget.observe(on_change1, names='value', type='change')
        


button_clear.on_click(on_button_clear_clicked)


In [7]:
# Define function to be called when manually moving the angle
def on_change_rot(change):
    _,_ = animate_curve(rot_deg=true_angle_widget.value)
    
# angle_widget.observe(on_change0, names='value', type='change')
true_angle_widget.observe(on_change_rot, names='value', type='change')
# type_widget.observe(on_change0, names='value', type='change')

In [8]:
# Here we make sure that true_angle_widget is updated each time angle_widget is modified
# This is because angle_widget is an integer used in the Play mode, and only true_angle_widget is linked to function rotating the line
def on_change_intangle(change):
    true_angle_widget.value = 0.1*angle_widget.value

angle_widget.observe(on_change_intangle, names='value', type='change')

In [9]:
# def on_change2(change):
#     angle_widget.value = true_angle_widget.value

# true_angle_widget.observe(on_change2, names='value', type='change')

In [10]:
plot_windmill.layout.min_width = '600px'
plot_windmill.layout.max_width = '600px'
plot_windmill.layout.min_height = '400px'
plot_windmill.layout.max_height = '400px'

play_button_angle = Play(min=0, max=7200, step=5, interval=20)
jslink((play_button_angle, 'value'), (angle_widget, 'value'))

# dl = widgets.dlink((angle_widget, 'value'), (true_angle_widget, 'value'))

display(Box([
        HBox([button_clear,play_button_angle,true_angle_widget], layout=Layout(justify_content='center',width='905px'))],
        layout=Layout(width='100%', flex_flow='row wrap', display='flex')))
display(HBox([plot_windmill]),layout=Layout(justify_content='center',width='905px'))

Box(children=(HBox(children=(Button(description='Clear selection!', style=ButtonStyle()), Play(value=0, interv…